Agent-based modeling of microglia

Andrew Gallimore, Enrique Lopez, Sela Raisl, Mack Kona, Anthony Wolfe and Kamila Larripa. 

California Polytechnic University, Humboldt

In [ ]:
import mesa
import numpy
import pandas